In [3]:
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)


In [ ]:
compile = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hi"},],
    temperature=0.0
)
print(compile.choices[0].message.content)

Hello! How can I assist you today?


### 데이터 전처리 코드

리뷰 JSON 파일 로드

In [1]:
import json

with open('./res/reviews.json', 'r') as f:
    review_list = json.load(f)
review_list[:3]

[{'review': '너무좋습니다 감사합니다', 'stars': 5, 'date': '2분 전'},
 {'review': '가족들과 좋은시간 보내고 왔습니다.\n부대시설 좋고 수영장도 좋습니다.\n앞으로도 종종 이용할 것 같습니다!\n\n날씨 좋을 때 한번 더 방문할께요!',
  'stars': 5,
  'date': '3시간 전'},
 {'review': '청결하고 좋았습니다.', 'stars': 5, 'date': '15시간 전'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개
- 나쁜 평점 = 별 4개 이하

In [2]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(189, 31)

좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [3]:
good_reviews, bad_reviews = [], []
for r in review_list:
    if r['stars'] == 5:
        good_reviews.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        bad_reviews.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

bad_reviews[:3]

['[REVIEW_START]시설적이 부분에서는 딱히 흠잡을게 없었고 가격대가 좀 있지만 퀄리티가 괜찮은 편이라 so so인데 유독 스파 직원들이 불친절함 표정도 그렇고 말투나 톤이 뭔가 하기 싫다는 듯한 느낌을 받음[REVIEW_END]',
 '[REVIEW_START]직원친절도 최상.  숙소 정비나 시설이 깨끗하고 뒷산뷰도 조용하고 좋았습니다.[REVIEW_END]',
 '[REVIEW_START]깨끗하고 산책하기도 좋아요. 애들이랑 물놀이하기 좋구요. 재미있게 놀다왔습니다.[REVIEW_END]']

In [4]:
good_reviews_text = '\n'.join(good_reviews)
bad_reviews_text = '\n'.join(bad_reviews)

전처리 함수 작성

In [5]:
import datetime
from dateutil import parser

def preprocess_reviews(path = './res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)

    reviews_good, reviews_bad = [], []
    current_date = datetime.datetime.now()
    date_bound = current_date - datetime.timedelta(days=6*30)

    for r in review_list:
        reviews_date_str = r['date']
        try:
            reviews_date = parser.parse(reviews_date_str)
        except(ValueError, TypeError):
            reviews_date = current_date
        
        if reviews_date < date_bound:
            continue

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)

    return reviews_good_text, reviews_bad_text

good, bad = preprocess_reviews()
print(good[:100])
print(bad[:100])

[REVIEW_START]너무좋습니다 감사합니다[REVIEW_END]
[REVIEW_START]가족들과 좋은시간 보내고 왔습니다.
부대시설 좋고 수영장도 좋습니다.
앞으로도 종종 
[REVIEW_START]시설적이 부분에서는 딱히 흠잡을게 없었고 가격대가 좀 있지만 퀄리티가 괜찮은 편이라 so so인데 유독 스파 직원들이 불친절함 표정도 그렇고 말투나 톤이 


### 평가 기준 설정
평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
    - 비교하는 방식 vs. 점수 매기는 방식
    - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
    - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summarise, accommodation)

In [ ]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion